# Montar Google Drive en Colab y cargar los archivos CSV

In [17]:
from google.colab import drive
import pandas as pd

# Montar Google Drive en Colab y cargar el archivo CSV.
drive.mount('/content/drive')

# Contiene los indicadores
file_path_1 = '/content/drive/My Drive/Colab Notebooks/checkpoint_2/data.csv'
df_1 = pd.read_csv(file_path_1, sep=';', encoding='utf-8')

# Contiene los códigos LAU de cada municipio
file_path_2 = '/content/drive/My Drive/Colab Notebooks/checkpoint_3/MadridLAUGeometries.csv'
df_2 = pd.read_csv(file_path_2, sep=';', encoding='utf-8')

# Contiene información sobre los indicadores
# file_path_3 = '/content/drive/My Drive/Colab Notebooks/checkpoint_3/indicadores.csv'
# df_3 = pd.read_csv(file_path_3, sep=';', encoding='utf-8')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


# Asociar código LAU a cada municipio

In [18]:
# Crear una copia de df_1
d1 = df_1.copy()

# Diccionario donde las claves son los municipios y los valores son los códigos
dict_code = df_2.set_index('lau_name')['code'].to_dict()

# Nueva columna 'lau_id' que mapea los nombres de los municipios a sus códigos
d1['lau_id'] = d1['Municipio'].map(dict_code)

# Obtener la lista de columnas
cols = list(d1.columns)

# Mover la columna 'lau_id' justo después de 'Municipio'
cols.insert(cols.index('Municipio')+1, cols.pop(cols.index('lau_id')))

# Reordenar el DataFrame y eliminar columnas sobrantes
d1 = d1[cols]
d1 = d1.drop(columns=['Unnamed: 0'])
d1.loc[d1[d1['ID'] == 1684].index, 'lau_id'] = 'ES_28168'


# Crear csv con valores

In [19]:
import re
import pandas as pd

indicadores = {
    2700: 'Cluster_Poblacion',
    2701: 'Cluster_Densidad',
    2702: 'Cluster_Distancia',

    2703: 'Variacion_pib_2015_Cluster_Poblacion',
    2704: 'Variacion_pib_2015_Cluster_Densidad',
    2705: 'Variacion_pib_2015_Cluster_Distancia',
    2706: 'Variacion_pib_2015_Zona_Estadistica',
    2707: 'Variacion_pib_2015_totalcomunidad',

    2708: 'Variacion_pib_capita_2015_Cluster_Poblacion',
    2709: 'Variacion_pib_capita_2015_Cluster_Densidad',
    2710: 'Variacion_pib_capita_2015_Cluster_Distancia',
    2711: 'Variacion_pib_capita_2015_Zona_Estadistica',
    2712: 'Variacion_pib_capita_2015_totalcomunidad',

    2713: 'Variacion_u_productivas_2015_Cluster_Poblacion',
    2714: 'Variacion_u_productivas_2015_Cluster_Densidad',
    2715: 'Variacion_u_productivas_2015_Cluster_Distancia',
    2716: 'Variacion_u_productivas_2015_Zona_Estadistica',
    2717: 'Variacion_u_productivas_2015_totalcomunidad',

    2718: 'Variacion_nempresas_total_2015_Cluster_Poblacion',
    2719: 'Variacion_nempresas_total_2015_Cluster_Densidad',
    2720: 'Variacion_nempresas_total_2015_Cluster_Distancia',
    2721: 'Variacion_nempresas_total_2015_Zona_Estadistica',
    2722: 'Variacion_nempresas_total_2015_totalcomunidad',

    2723: 'Variacion_nempresas_b_e_industria_2015_Cluster_Poblacion',
    2724: 'Variacion_nempresas_b_e_industria_2015_Cluster_Densidad',
    2725: 'Variacion_nempresas_b_e_industria_2015_Cluster_Distancia',
    2726: 'Variacion_nempresas_b_e_industria_2015_Zona_Estadistica',
    2727: 'Variacion_nempresas_b_e_industria_2015_totalcomunidad',

    2728: 'Variacion_nempresas_f_construccion_2015_Cluster_Poblacion',
    2729: 'Variacion_nempresas_f_construccion_2015_Cluster_Densidad',
    2730: 'Variacion_nempresas_f_construccion_2015_Cluster_Distancia',
    2731: 'Variacion_nempresas_f_construccion_2015_Zona_Estadistica',
    2732: 'Variacion_nempresas_f_construccion_2015_totalcomunidad',

    2733: 'Variacion_nempresas_comercio_transporte_hosteleria_2015_Cluster_Poblacion',
    2734: 'Variacion_nempresas_comercio_transporte_hosteleria_2015_Cluster_Densidad',
    2735: 'Variacion_nempresas_comercio_transporte_hosteleria_2015_Cluster_Distancia',
    2736: 'Variacion_nempresas_comercio_transporte_hosteleria_2015_Zona_Estadistica',
    2737: 'Variacion_nempresas_comercio_transporte_hosteleria_2015_totalcomunidad',

    2738: 'Variacion_nempresas_j_informacion_comunicaciones_2015_Cluster_Poblacion',
    2739: 'Variacion_nempresas_j_informacion_comunicaciones_2015_Cluster_Densidad',
    2740: 'Variacion_nempresas_j_informacion_comunicaciones_2015_Cluster_Distancia',
    2741: 'Variacion_nempresas_j_informacion_comunicaciones_2015_Zona_Estadistica',
    2742: 'Variacion_nempresas_j_informacion_comunicaciones_2015_totalcomunidad',

    2743: 'Variacion_nempresas_k_actividades_financieras_seguros_2015_Cluster_Poblacion',
    2744: 'Variacion_nempresas_k_actividades_financieras_seguros_2015_Cluster_Densidad',
    2745: 'Variacion_nempresas_k_actividades_financieras_seguros_2015_Cluster_Distancia',
    2746: 'Variacion_nempresas_k_actividades_financieras_seguros_2015_Zona_Estadistica',
    2747: 'Variacion_nempresas_k_actividades_financieras_seguros_2015_totalcomunidad',

    2748: 'Variacion_nempresas_l_actividades_inmobiliarias_2015_Cluster_Poblacion',
    2749: 'Variacion_nempresas_l_actividades_inmobiliarias_2015_Cluster_Densidad',
    2750: 'Variacion_nempresas_l_actividades_inmobiliarias_2015_Cluster_Distancia',
    2751: 'Variacion_nempresas_l_actividades_inmobiliarias_2015_Zona_Estadistica',
    2752: 'Variacion_nempresas_l_actividades_inmobiliarias_2015_totalcomunidad',

    2753: 'Variacion_nempresas_actividades_profesionales_tecnicas_2015_Cluster_Poblacion',
    2754: 'Variacion_nempresas_actividades_profesionales_tecnicas_2015_Cluster_Densidad',
    2755: 'Variacion_nempresas_actividades_profesionales_tecnicas_2015_Cluster_Distancia',
    2756: 'Variacion_nempresas_actividades_profesionales_tecnicas_2015_Zona_Estadistica',
    2757: 'Variacion_nempresas_actividades_profesionales_tecnicas_2015_totalcomunidad',

    2758: 'Variacion_nempresas_educacion_sanidad_servicios_sociales_2015_Cluster_Poblacion',
    2759: 'Variacion_nempresas_educacion_sanidad_servicios_sociales_2015_Cluster_Densidad',
    2760: 'Variacion_nempresas_educacion_sanidad_servicios_sociales_2015_Cluster_Distancia',
    2761: 'Variacion_nempresas_educacion_sanidad_servicios_sociales_2015_Zona_Estadistica',
    2762: 'Variacion_nempresas_educacion_sanidad_servicios_sociales_2015_totalcomunidad',

    2763: 'Variacion_nempresas_otros_servicios_personales_2015_Cluster_Poblacion',
    2764: 'Variacion_nempresas_otros_servicios_personales_2015_Cluster_Densidad',
    2765: 'Variacion_nempresas_otros_servicios_personales_2015_Cluster_Distancia',
    2766: 'Variacion_nempresas_otros_servicios_personales_2015_Zona_Estadistica',
    2767: 'Variacion_nempresas_otros_servicios_personales_2015_totalcomunidad',

    2768: 'Variacion_nempresas_total_servicios_2015_Cluster_Poblacion',
    2769: 'Variacion_nempresas_total_servicios_2015_Cluster_Densidad',
    2770: 'Variacion_nempresas_total_servicios_2015_Cluster_Distancia',
    2771: 'Variacion_nempresas_total_servicios_2015_Zona_Estadistica',
    2772: 'Variacion_nempresas_total_servicios_2015_totalcomunidad',
}

# Crear una lista para almacenar los valores de Codigo_Valor
codigo_valor_list = []

# Años a considerar
years = ['2015', '2016', '2017', '2018', '2019', '2020']

# Iterar sobre las columnas del DataFrame d1
for codigo, prefijo in indicadores.items():
    for year in years:
        col_name = f"{prefijo.replace('2015', year)}"
        if col_name in d1.columns:
            # Concatenar lau_id, código del indicador y año del indicador
            for lau_id in d1['lau_id']:
                codigo_valor = f"{lau_id}_{codigo}_{year}"
                codigo_valor_list.append(codigo_valor)

# Crear un nuevo DataFrame con la columna Codigo_Valor
df_codigo_valor = pd.DataFrame(codigo_valor_list, columns=['Codigo_Valor'])

# Crear las columnas adicionales requeridas
df_codigo_valor['Nombre'] = None
df_codigo_valor['Valor'] = None
df_codigo_valor['Year'] = None
df_codigo_valor['Indicador'] = None
df_codigo_valor['lau_id'] = None

# Iterar sobre las filas del DataFrame df_codigo_valor
for i, row in df_codigo_valor.iterrows():
    # Extraer lau_id, codigo y year del Codigo_Valor
    lau_id, codigo, year = re.match(r'(ES_\d{5})_(\d+)_(\d{4})', row['Codigo_Valor']).groups()
    # Asignar lau_id y year a las columnas correspondientes
    df_codigo_valor.at[i, 'lau_id'] = lau_id
    df_codigo_valor.at[i, 'Year'] = year
    df_codigo_valor.at[i, 'Indicador'] = codigo
    # Buscar el nombre del municipio en el DataFrame d1
    municipio = d1.loc[d1['lau_id'] == lau_id, 'Municipio'].values[0]
    df_codigo_valor.at[i, 'Nombre'] = municipio
    # Obtener el nombre del indicador correspondiente al código
    indicador = indicadores[int(codigo)]
    # Imprimir lau_id, codigo e indicador para depurar
    print(f"lau_id: {lau_id}, codigo: {codigo}, indicador: {indicador}, year: {year}")
    # Imprimir la consulta que se está realizando en el DataFrame d1
    print(f"Buscando en d1 con lau_id: {lau_id}, ID: {codigo}, columna: {indicador.replace('2015', year)}")
    # Buscar el valor del indicador en el DataFrame d1
    valor = d1.loc[d1['lau_id'] == lau_id, f"{indicador.replace('2015', year)}"].values[0]
    df_codigo_valor.at[i, 'Valor'] = valor


    """
    try:
        valor = d1.loc[(d1['lau_id'] == lau_id) & (d1['ID'] == codigo), f'{indicador}_{year}'].values[0]
        df_codigo_valor.at[i, 'Valor'] = valor
    except IndexError:
        print(f"No se encontró un valor para lau_id: {lau_id}, indicador: {indicador}, año: {year}")
        """


Se han truncado las últimas 5000 líneas del flujo de salida.
lau_id: ES_28146, codigo: 2770, indicador: Variacion_nempresas_total_servicios_2015_Cluster_Distancia, year: 2016
Buscando en d1 con lau_id: ES_28146, ID: 2770, columna: Variacion_nempresas_total_servicios_2016_Cluster_Distancia
lau_id: ES_28147, codigo: 2770, indicador: Variacion_nempresas_total_servicios_2015_Cluster_Distancia, year: 2016
Buscando en d1 con lau_id: ES_28147, ID: 2770, columna: Variacion_nempresas_total_servicios_2016_Cluster_Distancia
lau_id: ES_28150, codigo: 2770, indicador: Variacion_nempresas_total_servicios_2015_Cluster_Distancia, year: 2016
Buscando en d1 con lau_id: ES_28150, ID: 2770, columna: Variacion_nempresas_total_servicios_2016_Cluster_Distancia
lau_id: ES_28149, codigo: 2770, indicador: Variacion_nempresas_total_servicios_2015_Cluster_Distancia, year: 2016
Buscando en d1 con lau_id: ES_28149, ID: 2770, columna: Variacion_nempresas_total_servicios_2016_Cluster_Distancia
lau_id: ES_28151, codig

# Exportar csv

In [20]:
base_path = '/content/drive/My Drive/Colab Notebooks/checkpoint_3/'
d1.to_csv(base_path + 'data_lau.csv', sep=';', encoding='utf-8')

base_path = '/content/drive/My Drive/Colab Notebooks/checkpoint_3/'
df_codigo_valor.to_csv(base_path + 'valores.csv', sep=';', encoding='utf-8')
